Goal: Predict closing prices or detect potential trading signals.using lstm

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout ,Input ,LeakyReLU
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [ ]:
# df = pd.read_csv(file_path)
# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date', inplace=True)


In [ ]:
datasets_dir = os.path.join(os.getcwd(), 'datasets')
file_name = 'AABA_2006-01-01_to_2018-01-01.csv'
file_path = os.path.join(datasets_dir, file_name)
df = pd.read_csv(file_path)

In [ ]:
df.head()

In [ ]:
diff_close_series = df['Close'].diff().dropna()

dont make lag feature of actual value make lag feature of diffrences of the values


In [ ]:
lagged_data = pd.concat(
    [diff_close_series.shift(i) for i in range(5, -1, -1)],
    axis=1
).dropna()
lagged_data.columns = [f'lag_{i}' for i in range(5, -1, -1)]
X = lagged_data.iloc[:, 1:].values 
y = lagged_data.iloc[:, 0].values   

In [ ]:
# lag_df['diff t-1'] = lag_df['t'] - lag_df['t-1'] 
# lag_df['diff t'] = lag_df['t+1'] - lag_df['t']   

In [ ]:
# lag_df['rolling_mean_7'] = lag_df['t'].rolling(window=7).mean()
# lag_df['rolling_std_7'] = lag_df['t'].rolling(window=7).std()
# lag_df.dropna(inplace=True)

In [ ]:
# X = lag_df[['diff t-1', 't']].values  
# y = lag_df['t+1'].values

In [ ]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [ ]:

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

In [ ]:
# X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [ ]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [ ]:
# model = Sequential()
# model.add(LSTM(100, activation='relu', input_shape=(1, X_reshaped.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(50, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
print(lagged_data.head())
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_reshaped shape: {X_reshaped.shape}")
print(f"y_scaled shape: {y_scaled.shape}")

In [ ]:
model = Sequential()
model.add(Input(shape=(X_reshaped.shape[1], 1)))  # Explicit input layer
model.add(LSTM(50, return_sequences=False))
model.add(LeakyReLU(negative_slope=0.01))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# history = model.fit(X_reshaped, y_scaled, epochs=100, batch_size=16, verbose=1, validation_split=0.2)

In [ ]:
model.fit(X_reshaped, y_scaled, epochs=200, batch_size=8, verbose=0)

In [ ]:
y_pred_scaled = model.predict(X_reshaped)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

In [ ]:
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

In [ ]:
print(f"MSE: {mse:.6f}")
print(f"R2: {r2:.6f}")

In [145]:
n = X_reshaped.shape[0]  # Number of samples
k = X_reshaped.shape[1]  # Number of predictors (5 lags)
adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

In [146]:
print(f"Adjusted R2: {adjusted_r2:.6f}")

Adjusted R2: 0.861650


In [ ]:
print(f"Train samples: {X_reshaped.shape[0]}, Test samples: {X_test_reshaped.shape[0]}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y, label='Actual Differences')
plt.plot(y_pred, label='Predicted Differences')
plt.title("Close Price Differences: Actual vs Predicted")
plt.legend()
plt.show()

arima

In [ ]:
 split_idx = int(len(diff_close_series) * 0.8)
train, test = diff_close_series[:split_idx], diff_close_series[split_idx:]

In [ ]:
model_arima = ARIMA(train, order=(1, 0, 1))
model_arima_fit = model_arima.fit()

In [ ]:
forecast_arima = model_arima_fit.forecast(steps=len(test))

In [ ]:
mse_arima = mean_squared_error(test, forecast_arima)
mse_arima

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast_arima, label='ARIMA Forecast')
plt.title("ARIMA Forecast of Close Price Differences")
plt.legend()
plt.show()

In [ ]:
model_sarima = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 0, 1, 12))
model_sarima_fit = model_sarima.fit(disp=False)

In [ ]:
forecast_sarima = model_sarima_fit.forecast(steps=len(test))

In [ ]:
mse_sarima = mean_squared_error(test, forecast_sarima)
print(f"SARIMA MSE: {mse_sarima:.6f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast_sarima, label='SARIMA Forecast')
plt.title("SARIMA Forecast of Close Price Differences")
plt.legend()
plt.show()